<a href="https://colab.research.google.com/github/hyunjune123/Fake-News-Detection-Model/blob/main/fakenews_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
# 그냥 실행시킬 때 (나중에 java 환경변수 꼬일까봐 간단한걸로 바꿔놨엉)
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x 

+ pip install konlpy
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
     |████████████████████████████████| 460kB 43.5MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-

In [ ]:
import os, sys 
from google.colab import drive 
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from konlpy.tag import Mecab
from keras.preprocessing.text import Tokenizer
import pickle as pk
from keras.utils import np_utils
import numpy as np
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import load_model
from keras.regularizers import l2
import random
from sklearn.model_selection import train_test_split
import time
from scipy.stats import ks_2samp
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from pprint import pprint
import tensorflow as tf
import lightgbm as lgb

# Stopwords


In [ ]:
global stopwords
stops = ['도',
 "'",
 '1',
 '투자',
 '·',
 '2',
 '거래',
 '3',
 '정부',
 '&',
 '확인',
 '상',
 '최대',
 '추가',
 '하',
 'AI',
 '상품',
 '집중',
 '기반',
 '출시',
 '임',
 '신문',
 '밉다',
 '하나',
 '없이',
 '기회',
 '이유',
 '총선',
 '반',
 '모바일',
 '풀이',
 '빠르다',
 '주의',
 '주목',
 '최소',
 '입',
 '접수',
 '제주',
 '마감',
 '허가',
 '황',
 '포스',
 '제약',
 '키트',
 '포착',
 '파다',
 '무엇',
 '지점',
 '시티',
 '특허',
 '반드시',
 '즉시',
 '이던',
 '문자',
 '인가',
 '주도',
 '뉴딜',
 '최악',
 '신라',
 '풍',
 '어제',
 '개장',
 '한반도',
 '정말',
 '씨젠',
 '반전',
 '섹터',
 '넷',
 '500만원',
 '김정은',
 '펜',
 '동물',
 '외신',
 '왜',
 '익',
 '시선',
 '골든',
 '깜짝',
 '8억',
 '고수',
 '엄청나다',
 '링크',
 '1등',
 '중대',
 '9억',
 '자수',
 '윤석열',
 '지지율',
 '6억',
 '안철수',
 '지노',
 '결단',
 '세종시',
 '양해',
 '상상',
 '월요일',
 '날리다',
 '말고',
 '켐',
 '실전',
 '비밀',
 '다음주',
 '이만',
 '와중',
 '시마',
 '아나운서',
 '고백',
 '짜리',
 '폭등',
 '랠리',
 '냐',
 '빅텍',
 '입수',
 '폭염',
 '웰',
 '가세',
 '고맙다',
 '흔하다',
 'vs',
 '레이더',
 '년비',
 'EDGC',
 '미코',
 '리딩',
 '불륜',
 '심정',
 '하니',
 '무조건',
 '공부',
 '케이엠',
 '스탁',
 '아반떼',
 '안해',
 '우승',
 '충격파',
 '특허청',
 '9000만원',
 '막말',
 '겁니다',
 '짓',
 '혼자',
 '이야',
 'GV',
 '도대체',
 '17억',
 '->',
 '끼다',
 '지경',
 '시신',
 '레몬',
 '압',
 '몰아치다',
 'YBM',
 '아난',
 '참사',
 '차라리',
 '2000억',
 '속옷',
 '낫다',
 '북적',
 '솔젠트',
 '수젠텍',
 '페어',
 '전남도',
 '손학규',
 '비비안',
 '베테',
 '유니온',
 '제발',
 '15억',
 '압승',
 '찍',
 '민낯',
 '끔찍하다',
 '비트코인',
 '마침내',
 '16년',
 '초보',
 '성관계',
 'SKT',
 '유주',
 '兆',
 '몰락',
 '우린',
 '날로',
 '텔콘',
 'RF',
 '엠디',
 '곤두박질',
 '스톱',
 '기전',
 '제지',
 '068270',
 '명문제약',
 '과징금',
 '활발',
 '고비',
 '남영',
 '희귀',
 '삶다',
 '재테크',
 '부평',
 '납득',
 '베',
 '돌발',
 '찬사',
 '정경',
 '정봉주',
 '일진',
 '후기',
 ')*',
 '예요',
 '정현',
 '자마자',
 '모트',
 '065450',
 '200%',
 '도와주다',
 '출근길',
 '유비',
 'PI',
 '비말',
 '미투',
 '먹히다',
 '현산',
 '적나라하다',
 '무안군',
 '03',
 '톱텍',
 '비씨',
 '추월',
 '오늘이',
 'NE',
 '능률',
 '방위산업',
 '모처럼',
 '국동',
 '클리오',
 '열광하다',
 '길목',
 '알서포트',
 '비밀번호',
 '열차',
 '도이치',
 '몰락하다',
 'ABC',
 '광주대',
 '닉스',
 '005380',
 '떡',
 '035720',
 '엠씨',
 '+,',
 '열병',
 '건조기',
 '어나',
 '올인',
 'XX',
 '당혹',
 '알피',
 'WTO',
 '행방',
 '따라오다',
 '자가면역질환',
 '지온',
 '우디',
 '비엘',
 '정보보안',
 '와이즈',
 '올라타다',
 '국보법',
 '불능',
 '이명',
 '佛',
 '사이토카인',
 '양주',
 '신세',
 '대치동',
 '오스카',
 '쏘렌토',
 '대죄',
 '공중',
 '만능',
 '자살',
 '뺨',
 '전셋집',
 'STX',
 '전원주택',
 '퇴짜',
 '동동',
 '순천시',
 '(.)',
 '넨',
 '코스모신소재',
 '93%',
 '회계사',
 'PLUS',
 '휴장',
 '러시',
 '석권',
 '주성분',
 '감사원',
 '윈드',
 '차명진',
 '귀하다',
 '변종',
 '마그네슘',
 '이수진',
 '이정보',
 '의류건조기',
 'USMCA',
 '상폐',
 '풋',
 '확실',
 '028300',
 '클론',
 '더워지다',
 '당뇨병',
 ')/',
 '휴일',
 '여대',
 '여직원',
 '통쾌하다',
 '3~5',
 '발끈',
 '마니커',
 '무상증자',
 '일신동',
 '엑세스',
 '장대',
 '지누',
 '자명하다',
 '너비',
 '쌍끌이',
 '화천',
 '인턴십',
 '손흥민',
 '잡아가다',
 '넘어오다',
 '매점매석',
 '알콜',
 '흩어지다',
 '콜롬비아',
 '협',
 '섣부르다',
 '찬물',
 '맞춤화',
 '앵커',
 'NFC',
 '희비',
 '보성군',
 '16억',
 '힘스',
 '붙박다',
 '이장',
 '502',
 '50원',
 '女',
 '손대다',
 '넋',
 '맹공',
 '파다하다',
 '짝',
 '싸매다',
 '유희',
 '키위',
 '이광재',
 '0.46%',
 '코데즈',
 '컴바인',
 '에쓰오일',
 '화곡',
 '14.8%',
 '無',
 '68%',
 '196170',
 '005320',
 '98%',
 '007570',
 'MH',
 'MDS',
 '울상',
 '1200만원',
 'RFHIC',
 '지오',
 '용돈',
 '엠게임',
 '重',
 '영원',
 '2.20%',
 '쾌거',
 '억만장자',
 '700만원',
 '문희상',
 '수백억',
 'ASF',
 '유부',
 '이분',
 '변기',
 '서금',
 'ELB',
 '화순군',
 '223310',
 '베셀',
 '045060',
 '사지',
 'AK',
 '82억',
 '국제무역',
 '조광',
 '조아',
 '비화',
 '가상하다',
 '아가',
 '220만',
 '목돈',
 'SV',
 '디엠',
 '신생아',
 '3300만원',
 '보로',
 '버티고',
 '제임스',
 '1.07%',
 '비보',
 '서희',
 '183490',
 '올릭스',
 '집착',
 '대아',
 '키다리',
 '거둬들이다',
 '비적정',
 '서진',
 '44만원',
 'BIG',
 '퓨쳐',
 '소닉',
 '이티',
 '아모',
 '박은영',
 '동문',
 '단조',
 '블랙박스',
 '애국',
 '해킹',
 '먹구름',
 "'·'",
 '서장훈',
 '모자라다',
 '에버',
 '17.7%',
 '1.82%',
 '표창장',
 '작일',
 '25.9%',
 '5.00%',
 '홀짝',
 '모친',
 '고려아연',
 '7600원',
 '뭔',
 '보툴리눔톡신',
 '22억',
 '단군',
 '불명',
 '블록딜',
 '프로포폴',
 '꼬박꼬박',
 '74만원',
 '뿔',
 '문란하다',
 '술값',
 '낙하산',
 '단합',
 '도강',
 '9~10일',
 '2.76%',
 '상견례',
 '군사력',
 '동양네트웍스',
 '건양대',
 '이상민',
 '진도군',
 '삼국',
 '가스전',
 'IBD',
 'AM',
 '폴라리스',
 '177350',
 '시크릿',
 '세라믹',
 '방석',
 '004410',
 '복통',
 '硏',
 '034940',
 '24억원',
 '랩셀',
 '콘텐트',
 '어서',
 '격언',
 '131370',
 '쿼터',
 '17.5%',
 '獨',
 '102940',
 '파버나',
 '화일',
 '건드리다',
 '순방',
 'BJ',
 '라파스',
 '023150',
 '며느리',
 '서연',
 '여자친구',
 '동방',
 '세아',
 '노려보다',
 '나로',
 '맹물',
 '전자화폐',
 "...'",
 '9.85%',
 '0.82%',
 '0.68%',
 '0.24%',
 '처법',
 '폐해',
 '30원',
 '야말로',
 '점일',
 '3.09%',
 'ASCO',
 '신분당선',
 '구광모',
 '32억',
 '0.80%',
 '19.4%',
 '강간',
 '설마',
 '오리무중',
 'MP',
 '사그라들다',
 '석방',
 '뒤집혔다',
 '2050년',
 '22.2%',
 '아이엘',
 '사용권',
 '미용실',
 '티웨이',
 '21.2%',
 '뚝심',
 '23.3%',
 '제자리걸음',
 '이동성',
 'FnC',
 '31억',
 '분향소',
 '광림',
 'DLB',
 '고흥군',
 '\'..."',
 '2.54%',
 '옥석',
 '0.91%',
 '158억',
 '8:00',
 '1.40%',
 '습관화',
 '005490',
 '030520',
 '068050',
 '양동',
 '상신',
 '112240',
 '만이라도',
 '전초전',
 '벼락',
 '05',
 '108230',
 '025320',
 '006280',
 '013890',
 '042700',
 '6.37%',
 '284620',
 '089030',
 '털모자',
 '레퍼런스',
 '도일',
 '더라도',
 '청년실업',
 '039440',
 '242040',
 '잠도',
 '~"',
 '삼진',
 '234690',
 '신영',
 '등판',
 '0.25',
 '033180',
 '7.01%',
 '109%',
 '86조',
 '280억원',
 '046440',
 '구제역',
 'SH',
 '승주',
 '쉬쉬',
 '302550',
 'ITX',
 '289080',
 '컨택',
 '신도림',
 '102280',
 '033270',
 '000810',
 '럭슬',
 '김민경',
 '1.35%',
 '216080',
 '999',
 '016670',
 '57억',
 '쿼드',
 '만두',
 '캐리',
 '군사훈련',
 '0.56',
 '답보',
 '13,000원',
 '영진약품',
 '현혹',
 '제이스',
 '테판',
 '096690',
 '텍팜',
 '디알',
 '22.11',
 '2087.19',
 '2900만',
 '조성욱',
 '003620',
 '5.03%',
 '226950',
 '1000조',
 '軍',
 '000020',
 '메라',
 '재빠르다',
 '021050',
 '미미',
 '우전',
 '단자',
 '구출',
 '뉴보텍',
 '19조원',
 '포텐셜',
 '어게인',
 '0.87%',
 '김용식',
 '024950',
 '071950',
 '043200',
 '37분',
 '쪽방',
 '친모',
 '305',
 '2800원',
 '000990',
 '전조',
 '루멘',
 '권오갑',
 '090430',
 '008770',
 '나스',
 '패러디',
 '라고는',
 '친하다',
 '엇',
 '들것',
 '만가',
 '214270',
 '2004억',
 '자서전',
 '재혼',
 '덴트',
 '121800',
 '겨울방학',
 '우리동네',
 '238090',
 '891',
 '053050',
 '010660',
 '삼전',
 '리수',
 '19.8%',
 '026940',
 '권우',
 '048410',
 '010820',
 '5.16%',
 '수복',
 '4650원',
 '는커녕',
 '조로',
 '기독교인',
 '총독',
 '163',
 '별스탁',
 '이구나',
 '알베르토',
 '테기',
 '북핵',
 '지윤',
 '문래동',
 '290510',
 '035810',
 '001360',
 '260660',
 '베어링',
 '176440',
 '카일',
 '215380',
 '570',
 '크라이',
 '195990',
 '분분',
 '2400원',
 '우기홍',
 '유한양행',
 '복은',
 '056700',
 '26.0%',
 '192650',
 '052420',
 '지엘팜텍',
 '078520',
 '란제리',
 '몸매',
 '검정',
 '66억',
 '화훼단',
 '11억원',
 '160억',
 '3300원',
 '파트론',
 '091700',
 '기지개',
 '국유화',
 '박현빈',
 '친부',
 '초승달',
 '죽지',
 '늑장',
 '학생용',
 '후불',
 '6.85%',
 'KPS',
 '컬리지',
 '201',
 '422',
 '은커녕',
 '주례',
 '5천억',
 '출렁',
 '4.51',
 '0.79%',
 '1.59',
 '753.23',
 '1.43%',
 '\'.."',
 '참견',
 '514',
 '바티칸',
 '7.1',
 '노르',
 '23.78%',
 '2.34%',
 '온건',
 '월미도',
 '10원',
 '체리',
 '3.64',
 '5.23%',
 '휴켐스',
 '272억',
 '월풀',
 '환인제약',
 '印尼',
 '1.16',
 '퍼트',
 '스마트워크',
 '싱그럽다',
 '파이널',
 '퍼블리셔',
 '이글',
 '벳',
 '통큰',
 '2070',
 '29.91%',
 '렌털',
 '티원',
 '619억원',
 '편파',
 '26.2%',
 '애드',
 '11.57%',
 '김용호',
 '30조',
 '54.1%',
 '16.0%',
 '박영호',
 '마나',
 '워커',
 '거야',
 '이안',
 '1조달러',
 '1.51',
 '49.5',
 '꿋꿋',
 '패스트푸드',
 '명수',
 '스킨십',
 '김태영',
 '엠비언트',
 'Healthcare',
 '2.02%',
 '참좋은여행',
 '296억',
 '모터스포츠',
 '7.14%',
 '제일고',
 '판지',
 'iF',
 '가뭄',
 '성문',
 '체시스',
 '경상수지',
 '화서역',
 '투셀',
 '중고생',
 '1950',
 '파혼',
 '우림',
 '91%',
 '남아공',
 '691.53',
 '375',
 '2.43%',
 '김영희',
 '컨템포러리',
 '전기자동차',
 '2900억',
 '사이니',
 '송가',
 '황창규',
 '헬로비전',
 '킨텍스',
 '화끈',
 '폼팩터',
 '직사각형',
 '3.18%',
 '51.0%',
 '이정표',
 'XM',
 '지망',
 '18.57%',
 '스페코',
 '크론병',
 '양행',
 '달려들다',
 '콧대',
 '괴정동',
 '9500만원',
 '28.2%',
 '지엔',
 '예사롭다',
 '난타',
 '1146',
 '논객',
 '사직서',
 '49만',
 '1890',
 '축협',
 '의외로',
 '조현병',
 '독도',
 '가만',
 '후진',
 '친일파',
 '술자리',
 '기프티콘',
 '빅딜',
 '시커멓다',
 '무방비',
 '번화가',
 '1.6만',
 '묵적',
 '송중기',
 '강호동',
 '스터디',
 '텅텅',
 '뱉다',
 '4500',
 '상호주',
 '화채',
 '배종옥',
 '봄바람',
 '뒷전',
 '샛별',
 '기에만',
 '들끓다',
 '132억',
 '설왕설래',
 '격분',
 '수요집회',
 '맞대다',
 '동복',
 '브로',
 '여진',
 '150억',
 '가관',
 '콩팥',
 '26.1%',
 '발명',
 '2025',
 '정갑영',
 '쑥대밭',
 '연예기획사',
 '떡볶이',
 '파랗다',
 '연석',
 '아버님',
 '친노',
 '737억',
 '23.6%',
 '반성은',
 '720',
 '15.2%',
 'FPD',
 '54억원',
 '12.16',
 '코앞',
 '인증샷',
 '289억',
 '399억',
 '#제일바이오',
 '1.93',
 '644.93',
 '윤호중',
 '통용',
 'KC',
 '랜드로버',
 '오프로드',
 'TD',
 '6837만원',
 '김정화',
 '창정',
 '기다림',
 '멜라',
 '아르',
 '136',
 '비철금속',
 '166억원',
 '함진규',
 '9.54%',
 '원색',
 '151만',
 '56조',
 '허구',
 '189억',
 '6700억',
 '태평',
 '폭행죄',
 '6.35%',
 '무상교육',
 '스마트워치',
 '1629',
 '무니',
 '편안',
 '경산시',
 '조선족',
 '3400',
 '출시일',
 '1.2조',
 'ISA',
 '현충일',
 '프랜',
 '김기문',
 '점령',
 '의외',
 '웰크론한텍',
 '1918년',
 '분구']

# Used funtions

In [ ]:
### drive mount ###

def driveMount():
  global base_path
  drive.mount('/content/mnt')
  base_path = "/content/mnt/My Drive/Datacon/Fake_news/"


### Data process ###

# get data
def getData():
  global train_data, test_data
  train_data = pd.read_csv(base_path + "news_train.csv")[['title', 'content','info']]
  test_data = pd.read_csv(base_path + "news_test.csv")[['title', 'content']]
  df = train_data
  true_df = df[df['info'] == 1]
  false_df = df[df['info'] == 0]
  print(len(false_df))
  sampleIndices = [i for i in range(len(false_df))]
  indices = random.sample(sampleIndices, k = len(true_df))
  new_false_df = false_df.iloc[indices, :]
  train_data = shuffle(new_false_df.append(true_df, ignore_index=True))



# tokenize
def makeTokenize():
  global X_train_tokenized, X_test_tokenized
  okt = Okt()
  mecab = Mecab()

  
  #stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', 
  #             '도', '를', '으로', '자', '에', '와', '한', '하다'] + stops

  X_train_tokenized = []
  text = list(train_data['content'])

  for i in tqdm(range(len(text))):
    temp_X = []
    temp_X = okt.morphs(text[i], stem=True) # 토큰화
    #temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1]
    X_train_tokenized.append(temp_X)

  X_test_tokenized = []
  text = list(test_data['content'])

  for i in tqdm(range(len(text))):
    temp_X = []
    temp_X = okt.morphs(text[i], stem=True) # 토큰화
    #temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    temp_X = [word for word in temp_X if len(word) > 1]
    X_test_tokenized.append(temp_X)
  with open(base_path+'X_train_tokenized.pkl', 'wb') as f:
    pk.dump(X_train_tokenized, f)
  with open(base_path+'X_test_tokenized.pkl', 'wb') as f:
    pk.dump(X_test_tokenized, f)

# integer encoding
def makeIntegerEncoding(max_words=70000):
  global X_train_encoded,X_test_encoded
  tokenizer = Tokenizer(num_words = max_words)
  tokenizer.fit_on_texts(X_train_tokenized+X_test_tokenized)
  print("unique tokens : %d" %len(tokenizer.word_index))
  X_train_encoded = tokenizer.texts_to_sequences(X_train_tokenized)
  X_test_encoded = tokenizer.texts_to_sequences(X_test_tokenized)
  with open(base_path+'X_train_encoded.pkl', 'wb') as f:
    pk.dump(X_train_encoded, f)
  with open(base_path+'X_test_encoded.pkl', 'wb') as f:
    pk.dump(X_test_encoded, f)

def setMaxLength(max_len=200):
  global X_train_fixed,X_test_fixed
  X_train_fixed = pad_sequences(X_train_encoded, maxlen=max_len)  # X_train data 최종형태
  X_test_fixed = pad_sequences(X_test_encoded, maxlen=max_len) # X_test data 최종형태
  # save final vectors
  with open(base_path+'X_train.pkl', 'wb') as f:
    pk.dump(X_train_fixed, f)
  with open(base_path+'X_test.pkl', 'wb') as f:
    pk.dump(X_test_fixed, f)

def oneHotEncoding():
  global y_train  # y_train data 최종형태
  y_train = []

  for i in range(len(train_data['info'])):
    if train_data['info'].iloc[i] == 1:
      y_train.append([0, 1])
    elif train_data['info'].iloc[i] == 0:
      y_train.append([1, 0])

  y_train = np.array(y_train)
  # save final vectors
  with open(base_path+'y_train.pkl', 'wb') as f:
    pk.dump(y_train, f)


# load integer vectors
def loadIntegerVectors():
  global X_train_fixed, X_test_fixed, y_train_fixed, X_train_encoded, X_test_encoded, X_train_tokenized, X_test_tokenized

  with open(base_path+'X_train.pkl', 'rb') as f:
	  X_train_fixed = pk.load(f)
   
  with open(base_path+'X_test.pkl', 'rb') as f:
	  X_test_fixed = pk.load(f)
   
  with open(base_path+'y_train.pkl', 'rb') as f:
    y_train_fixed = pk.load(f)

  with open(base_path+'X_train_encoded.pkl', 'rb') as f:
    X_train_encoded = pk.load(f)

  with open(base_path+'X_test_encoded.pkl', 'rb') as f:
    X_test_encoded = pk.load(f) 

  with open(base_path+'X_train_tokenized.pkl', 'rb') as f:
    X_train_tokenized = pk.load(f)

  with open(base_path+'X_test_tokenized.pkl', 'rb') as f:
    X_test_tokenized = pk.load(f) 



### make Submit Data ###
def makeSubmitFile(model,modelName, testSet):
  outputname = modelName + ".csv"
  mypredict = model.predict(testSet)

  test_data_forsubmit = pd.read_csv(base_path + "news_test.csv")
  submission_ids = list(test_data_forsubmit['id'])
  my_result = []

  for i in range(len(mypredict)):
    print(mypredict[i])
    my_result.append(int(round(mypredict[i][0])))

  sub_dict = {"id":submission_ids, "info":my_result}
  submission_df = pd.DataFrame(sub_dict)
  submission_df.to_csv(outputname, index=False, encoding='utf-8')
  pd.read_csv(outputname)


### Print Data Distribution ###

def printDataDistribution():
  train_data['info'].value_counts().plot(kind='bar', title = 'train data label distribution')
  print(train_data.groupby('info').size().reset_index(name='count'))

def printDataLength_train(x):
  len_vec = sorted([len(l) for l in x])
  print("가장 긴 문자 길이 40 :", len_vec[len(len_vec)-40:len(len_vec)])
  print("문자의 최대 길이 :" , len_vec[len(len_vec)-1])
  print("문자의 평균 길이 : ", sum(len_vec)/ len(x))
  plt.hist([len_vec], bins=50)
  plt.xlabel('length of Data')
  plt.ylabel('number of Data')
  plt.title("Train line integer eoncoding length distribution")
  plt.show()

def printDataLength_test():
  len_vec = sorted([len(l) for l in X_test])
  print("가장 긴 문자 길이 40 :", len_vec[len(len_vec)-40:len(len_vec)])
  print("문자의 최대 길이 :" , len_vec[len(len_vec)-1])
  print("문자의 평균 길이 : ", sum(len_vec)/ len(X_test))
  plt.hist([len_vec], bins=50)
  plt.xlabel('length of Data')
  plt.ylabel('number of Data')
  plt.title("Test line integer eoncoding length distribution")
  plt.show()


In [ ]:
# 처음 벡터를을 저장하는 경우
def saveIntegerVector():
  driveMount()
  getData()
  makeTokenize()
  makeIntegerEncoding()
  setMaxLength()
  oneHotEncoding()
    

In [ ]:
saveIntegerVector()

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


  0%|          | 0/93864 [00:00<?, ?it/s]

71813


100%|██████████| 142565/142565 [09:37<00:00, 246.99it/s]


unique tokens : 64820


In [ ]:
driveMount()
loadIntegerVectors()
makeIntegerEncoding()
setMaxLength()
oneHotEncoding()


Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).
unique tokens : 63759


# DNN


In [ ]:
from sklearn.metrics import accuracy_score

MAX_SEQUENCE_LENGTH = 200
MAX_NUM_WORDS = 70000
EMBEDDING_DIM = 500
TEST_SPLIT = 0.2

def evaluate_model(predict_fun, X_train, y_train, X_test, y_test):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    # training error
    y_predict_train = predict_fun(X_train)
    train_acc = accuracy_score(y_train,y_predict_train)
    
    # testing error
    y_predict_test = predict_fun(X_test)
    test_acc = accuracy_score(y_test,y_predict_test)
    
    return train_acc, test_acc

loadIntegerVectors()
print('Shape of data tensor:', X_train_fixed.shape)
print('Shape of label tensor:', y_train_fixed.shape)



Shape of data tensor: (93864, 200)
Shape of label tensor: (93864, 2)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X_train_fixed, 
                                                  np.array([y[0]*0+y[1]*1 for y in y_train_fixed]), 
                                                  test_size=TEST_SPLIT)

print('Shape of train tensor:', x_train.shape)
print('Shape of label tensor:', y_train.shape)
print('Shape of val train tensor:', x_val.shape)
print('Shape of val label tensor:', y_val.shape)

Shape of train tensor: (75091, 200)
Shape of label tensor: (75091,)
Shape of val train tensor: (18773, 200)
Shape of val label tensor: (18773,)


In [ ]:
# DNN MODEL

from keras import layers
from keras.models import Sequential
model = Sequential(
    [
        # part 1: word and sequence processing
        layers.Embedding(MAX_NUM_WORDS,
                         EMBEDDING_DIM, 
                         input_length=MAX_SEQUENCE_LENGTH,
                         trainable=True),
        layers.Conv1D(128, 5, activation='relu'),
        layers.GlobalMaxPooling1D(),
        
        # part 2: classification
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 500)          35000000  
_________________________________________________________________
conv1d (Conv1D)              (None, 196, 128)          320128    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 35,336,769
Trainable params: 35,336,769
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train the model

es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

history = model.fit(x_train, 
                    y_train,
                    batch_size=512,
                    epochs=3,
                    validation_data=(x_val, y_val),
                    callbacks = [es])

Epoch 1/3
147/147 [==============================] - 605s 4s/step - loss: 0.1957 - accuracy: 0.9256 - val_loss: 0.0724 - val_accuracy: 0.9721
Epoch 2/3
147/147 [==============================] - 611s 4s/step - loss: 0.0341 - accuracy: 0.9881 - val_loss: 0.0543 - val_accuracy: 0.9791
Epoch 3/3
 67/147 [============>.................] - ETA: 5:21 - loss: 0.0100 - accuracy: 0.9972

In [ ]:
# evaluate model

def predict(X):
    return np.rint(model.predict(X)) # threshold the predictions to retrieve labels

train_acc, test_acc = evaluate_model(predict,
                                     x_train, 
                                     y_train, 
                                     x_val, 
                                     y_val)
print("Training Accuracy: {:.2f}%".format(train_acc*100))
print("Testing Accuracy: {:.2f}%".format(test_acc*100))

Training Accuracy: 99.61%
Testing Accuracy: 97.60%
